From `analysis_notebooks/R68_eff_plot.ipynb`

In [1]:
#Import libraries & such

from IPython.core.display import display, HTML
from scipy.optimize import curve_fit
display(HTML("<style>.container { width:100% !important; }</style>"))

exec(open("../analysis_notebooks/nb_setup.py").read()) #Is there a better way to do this?

from matplotlib import *
style.use('../mplstyles/stylelib/standard.mplstyle')

from R68_efficiencies import *

../mplstyles
3.4.2
/mnt/d/git/R68_paper2019/mplstyles


ModuleNotFoundError: No module named 'R68_efficiencies'

In [ ]:
E_thresh=50 #[eV] Possible analysis threshold
E_test=np.logspace(-5,3.5,2000)

fig_w=7

In [ ]:
#Cut efficiencies
#fig,ax = plt.subplots(1,1,figsize=(16,12),sharex=True)
fig,ax = plt.subplots(1,1)

ax.axhline(eff_tail, color = next(ax._get_lines.prop_cycler)['color'], label='ctail')
ax.axhspan(eff_tail-deff_tail, eff_tail+deff_tail, alpha=0.5)

ax.axhline(eff_pileup, color = next(ax._get_lines.prop_cycler)['color'], label='cpileup ')
ax.axhspan(eff_pileup-deff_pileup, eff_pileup+deff_pileup, alpha=0.5)

line,=ax.plot(E_test, spikeEff(E_test), label='cSpikey')
ax.fill_between(E_test, spikeEff(E_test)-dspikeEff(E_test)[1], spikeEff(E_test)+dspikeEff(E_test)[0], 
                alpha=0.5, color=line.get_color())

line,=ax.plot(E_test,chisqEff(E_test), label='cchit')
ax.fill_between(E_test, chisqEff(E_test)-dchisqEff(E_test)[1], chisqEff(E_test)+dchisqEff(E_test)[0], 
                alpha=0.5, color=line.get_color())

line,=ax.plot(E_test, cutEff(E_test), label='Total')
ax.fill_between(E_test, cutEff(E_test)-dcutEff(E_test)[1], cutEff(E_test)+dcutEff(E_test)[0], 
                alpha=0.5, color=line.get_color())

line,=ax.plot(E_test, trigburstEff(E_test), label='ctrigburst')
ax.fill_between(E_test, trigburstEff(E_test)-dtrigburstEff(E_test)[1], trigburstEff(E_test)+dtrigburstEff(E_test)[0], 
                alpha=0.5, color=line.get_color())

ax.axvline(E_thresh, color='m', linestyle='--', linewidth=2, label='Threshold')

#Fit a smooth function to total efficiency
def effFit_func(x,x0,sigma,a,b):
    return (x>x0)*(a+b*x)*(1-np.exp(-(x-x0)/sigma))
Efitmin=50
Efitmax=1000
cfit=(E_test>=Efitmin) & (E_test<Efitmax)
popt,pcov=curve_fit(effFit_func,E_test[cfit], cutEff(E_test)[cfit],p0=[30,30,0.45,1e-4])

#Set a single uncertainty for the fit to encompass all the 1-sigma discrepancies in the measured curve
maxneg=np.max(np.abs( effFit_func(E_test,*popt) - (cutEff(E_test)-dcutEff(E_test)[1]))[cfit])
maxpos=np.max(np.abs( effFit_func(E_test,*popt) - (cutEff(E_test)+dcutEff(E_test)[0]))[cfit])

dEffFit=max(maxneg,maxpos)

def cutEffFit(E):
    eff=np.ones_like(E)
    eff[(E>=50)&(E<1000)]=effFit_func(E[(E>=50)&(E<1000)], 2.16419228e+01, 1.61938240e+01, 4.28262762e-01, 3.17895378e-05)
    eff[E>=1000]=0.51633498
    return eff

def dcutEffFit(E):
    deff=np.zeros_like(E)
    deff[(E>=50)&(E<1000)]=0.02975340974950752
    deff[E>=1000]=0.01294113
    return deff

ax.plot(E_test[E_test>Efitmin],cutEffFit(E_test[E_test>Efitmin]),'k',label='Cut Efficiency Fit')
ax.fill_between(E_test[E_test>Efitmin],(cutEffFit(E_test)-dcutEffFit(E_test))[E_test>Efitmin],(cutEffFit(E_test)+dcutEffFit(E_test))[E_test>Efitmin],color='k',alpha=0.2)

fig.suptitle('R68 PuBe Cut Efficiencies')
ax.set_xlim(30,2e3)
ax.set_xscale('log')
ax.set_ylim(0,1.05)
ax.set_xlabel(r'$E_{recoil}$ [eV]')
ax.set_ylabel('Efficiency')
ax.legend(ncol=2)

plt.tight_layout()

plt.savefig('figures/cut_effs.pdf')
plt.show()

In [ ]:
#Cut efficiencies for bkg data

fig,ax = plt.subplots(1,1,figsize=(9.0,8.0),sharex=True)


ax.axhline(eff_tail_bkg, color = next(ax._get_lines.prop_cycler)['color'], label='ctail')
ax.axhspan(eff_tail_bkg-deff_tail_bkg, eff_tail_bkg+deff_tail_bkg, alpha=0.5)

ax.axhline(eff_pileup_bkg, color = next(ax._get_lines.prop_cycler)['color'], label='cpileup ')
ax.axhspan(eff_pileup_bkg-deff_pileup_bkg, eff_pileup_bkg+deff_pileup_bkg, alpha=0.5)



line,=ax.plot(E_test, spikeEff_bkg(E_test), label='cSpikey')
ax.fill_between(E_test, spikeEff_bkg(E_test)-dspikeEff_bkg(E_test)[1], spikeEff_bkg(E_test)+dspikeEff_bkg(E_test)[0], 
                alpha=0.5, color=line.get_color())

line,=ax.plot(E_test,chisqEff_bkg(E_test), label='cchit')
ax.fill_between(E_test, chisqEff_bkg(E_test)-dchisqEff_bkg(E_test)[1], chisqEff_bkg(E_test)+dchisqEff_bkg(E_test)[0], 
                alpha=0.5, color=line.get_color())

line,=ax.plot(E_test, cutEff_bkg(E_test), label='Total')
ax.fill_between(E_test, cutEff_bkg(E_test)-dcutEff_bkg(E_test)[1], cutEff_bkg(E_test)+dcutEff_bkg(E_test)[0], 
                alpha=0.5, color=line.get_color())

next(ax._get_lines.prop_cycler)['color']

ax.axvline(E_thresh, color='m', linestyle='--', linewidth=2, label='Threshold')

#Fit with smooth function
Efitmin=50
popt,pcov=curve_fit(effFit_func,E_test[E_test>Efitmin], cutEff_bkg(E_test)[E_test>Efitmin],p0=[30,30,0.45,1e-4])

#Set a single uncertainty for the fir to encompass all the 1-sigma discrepancies in the measured curve
maxneg=np.max(np.abs( effFit_func(E_test,*popt) - (cutEff_bkg(E_test)-dcutEff_bkg(E_test)[1]))[E_test>Efitmin])
maxpos=np.max(np.abs( effFit_func(E_test,*popt) - (cutEff_bkg(E_test)+dcutEff_bkg(E_test)[0]))[E_test>Efitmin])
dEffFit_bkg=max(maxneg,maxpos)

def cutEffFit_bkg(E):
    return effFit_func(E, 4.23508507e+01,  7.61590223e+00,  7.14511426e-01, -5.10367727e-06)
def dcutEffFit_bkg(E):
    return 0.16643841685131444

ax.plot(E_test[E_test>Efitmin],cutEffFit_bkg(E_test[E_test>Efitmin]),'k',label='Cut Efficiency Fit')
ax.fill_between(E_test[E_test>Efitmin],(cutEffFit_bkg(E_test)-dcutEffFit_bkg(E_test))[E_test>Efitmin],(cutEffFit_bkg(E_test)+dcutEffFit_bkg(E_test))[E_test>Efitmin],color='k',alpha=0.2)

fig.suptitle('R68 Bkg Cut Efficiencies')
ax.set_xlim(30,2e3)
ax.set_xscale('log')
ax.set_ylim(0,1.05)
ax.set_xlabel(r'$E_{recoil}$ [eV]')
ax.set_ylabel('Efficiency')
ax.legend(ncol=2)

#plt.tight_layout()

plt.savefig('figures/cut_effs_bkg.pdf')
plt.show()